In [0]:
# Importando as bibliotecas necessárias
from delta.tables import *
# Caminho para o diretório Delta Lake onde os dados serão armazenados
delta_path = "/path/evolutivo"

# Criação de um DataFrame de exemplo com um esquema inicial
df = spark.createDataFrame([(1, "João", 25), (2, "Maria", 30)], ["id", "nome", "idade"])
df.show()

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1| João|   25|
|  2|Maria|   30|
+---+-----+-----+



In [0]:
# Salvando o DataFrame no formato Delta Lake
df.write.format("delta").mode("overwrite").save(delta_path)

# Realizando uma atualização no esquema: adicionando uma nova coluna "cidade"
df_novo_esquema = spark.createDataFrame([(1, "João", 25, "São Paulo"), (2, "Maria", 30, "Rio de Janeiro")],
                                        ["id", "nome", "idade", "cidade"])

# Salvando o DataFrame com o novo esquema no mesmo diretório Delta Lake
df_novo_esquema.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(delta_path)

# Carregando o DeltaTable a partir do caminho especificado
delta_table = DeltaTable.forPath(spark, delta_path)

# Visualizando as versões disponíveis
delta_table.history().show(truncate=False)

+-------+-------------------+---------------+---------------------------+---------+--------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId         |userName                   |operation|operationParameters                   |job |notebook          |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-------------------+---------------+---------------------------+---------+--------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1      |2024-03-24 14:17:11|18226580

In [0]:
# Lendo o DataFrame com o esquema evolutivo
df_esquema_atual = spark.read.format("delta").load(delta_path)
# Exibindo o DataFrame resultante
df_esquema_atual.show()

+---+-----+-----+--------------+
| id| nome|idade|        cidade|
+---+-----+-----+--------------+
|  2|Maria|   30|Rio de Janeiro|
|  1| João|   25|     São Paulo|
+---+-----+-----+--------------+



In [0]:
# Carregando o DataFrame de uma versão anterior do Delta Lake
df_versao_0 = spark.read.format("delta").option("versionAsOf", 0).load(delta_path)
# Exibindo o DataFrame da versão 0
df_versao_0.show()

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1| João|   25|
|  2|Maria|   30|
+---+-----+-----+



In [0]:
dbutils.fs.rm("/path/evolutivo", recurse=True)

Out[5]: True